In [1]:
import torch
import pandas as pd
import re
import spacy
from torchtext.data.utils import get_tokenizer
import torch.nn as nn
import torch.nn.functional as F
import random
import torch.optim as optim
import torchtext
import time

In [2]:
from models import LSTMNet
import site
import os
os.environ['SP_DIR'] = '/opt/conda/lib/python3.11/site-packages'

accuracies = []

# nlp library of Pytorch
from torchtext import data

import warnings as wrn
wrn.filterwarnings('ignore')


SEED = 2021

torch.manual_seed(SEED)
torch.backends.cuda.deterministic = True

In [3]:
# def clean_text(text):
#     cleaned_text = re.sub(r'[^A-Za-z0-9]+', ' ', str(text))
#     return cleaned_text

# # Load and preprocess the data files
# def load_and_preprocess(file_path):
#     df = pd.read_csv(file_path, header=None, delimiter='\t') # Assuming tab-separated values in .data files
#     df[1] = df[1].apply(clean_text) # Assuming the text is in the second column
#     cleaned_file_path = file_path.replace('.data', '_cleaned.data')
#     df.to_csv(cleaned_file_path, index=False, header=False)
#     return cleaned_file_path

In [4]:
# cleaned_train_file = load_and_preprocess('./data/ag_news/data/train.data')
# cleaned_valid_file = load_and_preprocess('./data/ag_news/data/valid.data')
# cleaned_test_file = load_and_preprocess('./data/ag_news/data/test.data')

In [5]:
cleaned_train_file = 'data/ag_news/train_clean.csv'
cleaned_valid_file = 'data/ag_news/valid_clean.csv'
cleaned_test_file = 'data/ag_news/test_clean.csv'

In [6]:
spacy_en = spacy.load('en_core_web_sm')

def spacy_tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

LABEL = data.LabelField()
TEXT = data.Field(tokenize=spacy_tokenizer, batch_first=True, include_lengths=True)
fields = [("label", LABEL), ("text", TEXT)]

training_data = data.TabularDataset(path=cleaned_train_file, format="csv", fields=fields, skip_header=True)
validation_data = data.TabularDataset(path=cleaned_valid_file, format="csv", fields=fields, skip_header=True)
test_data = data.TabularDataset(path=cleaned_test_file, format="csv", fields=fields, skip_header=True)

print(vars(training_data.examples[0]))

train_data,valid_data = training_data, validation_data

TEXT.build_vocab(train_data,
                 min_freq=5)

LABEL.build_vocab(train_data)
# Count the number of instances per class
label_counts = {LABEL.vocab.itos[i]: LABEL.vocab.freqs[LABEL.vocab.itos[i]] for i in range(len(LABEL.vocab))}
print("Number of instances per class:", label_counts)


print("Size of text vocab:",len(TEXT.vocab))

print("Size of label vocab:",len(LABEL.vocab))

TEXT.vocab.freqs.most_common(10)

# Creating GPU variable
device = torch.device("cuda")
#device = torch.device('cuda')
print(f'Using device: {device}')


Sent_SIZE=32
print("Batch size initialized")
calib_data=train_data


{'label': '4', 'text': ['Apple', 'yesterday', 'released', 'a', '12', '7', 'MB', 'security', 'update', 'that', 'consists', 'of', 'several', 'revised', 'components', 'including', 'Apache', 'AppKit', 'HIToolbox', 'Kerberos', 'Postfix', 'PSNormalizer', 'Safari', 'and', 'Terminal']}
Number of instances per class: {'2': 30000, '3': 30000, '4': 30000, '1': 29999}
Size of text vocab: 27797
Size of label vocab: 4
Using device: cuda
Batch size initialized


In [7]:
LABEL.build_vocab(calib_data)
# Count the number of instances per class
label_counts = {LABEL.vocab.itos[i]: LABEL.vocab.freqs[LABEL.vocab.itos[i]] for i in range(len(LABEL.vocab))}
print("Number of instances per class:", label_counts)

Number of instances per class: {'2': 30000, '3': 30000, '4': 30000, '1': 29999}


In [8]:
train_iterator,validation_iterator = data.BucketIterator.splits(
    (train_data,valid_data),
    batch_size = Sent_SIZE,
    # Sort key is how to sort the samples
    sort_key = lambda x:len(x.text),
    sort_within_batch = True,
    device = device
)

test_iterator = data.BucketIterator(
    test_data,
    batch_size=Sent_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    device=device
)

In [9]:
SIZE_OF_VOCAB = len(TEXT.vocab)
EMBEDDING_DIM = 100
NUM_HIDDEN_NODES = 100
NUM_OUTPUT_NODES = len(LABEL.vocab)
NUM_LAYERS = 1
BIDIRECTION = False
DROPOUT = 0.2
BIT_WIDTH = 3

In [10]:
print(SIZE_OF_VOCAB)
print(NUM_OUTPUT_NODES)

27797
4


In [11]:
model = LSTMNet(SIZE_OF_VOCAB, EMBEDDING_DIM, NUM_HIDDEN_NODES, NUM_OUTPUT_NODES, NUM_LAYERS, BIDIRECTION, DROPOUT, BIT_WIDTH).to(device)
criterion = nn.CrossEntropyLoss()

print(torch.cuda.is_available())

model = model.to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-3)

model

True


LSTMNet(
  (embedding): Embedding(27797, 100)
  (lstm): LSTM(100, 100, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=4, bias=False)
)

In [12]:
def multi_class_accuracy(preds, y):
    _, predicted = torch.max(preds, 1)
    correct = (predicted == y).float()
    acc = correct.sum() / len(correct)
    return acc

def evaluate(model,iterator,criterion):
    
    epoch_loss = 0.0
    epoch_acc = 0.0
    
    # deactivate the dropouts
    model.eval()
    
    # Sets require_grad flat False
    with torch.no_grad():
        for batch in iterator:
            text,text_lengths = batch.text
            text, text_lengths = text.to(device), text_lengths.to(device)
            batch_label = batch.label.to(device)
            predictions = model(text,text_lengths).squeeze()
              
            #compute loss and accuracy
            loss = criterion(predictions, batch_label)
            acc = multi_class_accuracy(predictions, batch_label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [13]:
# Create dataset and iterator for test data
test_fields = [("label", LABEL), ("text", TEXT)]
test_data = data.TabularDataset(path=cleaned_test_file, format="csv", fields=test_fields, skip_header=True)

In [14]:
device = torch.device("cpu")
model=model.to(device)
for name, param in model.named_parameters():
    print(name)

embedding.weight
lstm.weight_ih_l0
lstm.weight_hh_l0
lstm.bias_ih_l0
lstm.bias_hh_l0
fc.weight


In [15]:
from brevitas.graph.quantize import preprocess_for_quantize
from ptq_common import quantize_model, apply_bias_correction, apply_act_equalization

In [16]:
pre_model = preprocess_for_quantize(
            model,
            equalize_iters=20,
            equalize_merge_bias=True,
            merge_bn=True,
            channel_splitting_ratio=0.0,
            channel_splitting_split_input=False)

In [17]:
dtype = getattr(torch, 'float')

In [18]:
print(device)
quant_model = quantize_model(
        pre_model.to(device),
        dtype=dtype,
        device=device,
        backend='layerwise',
        scale_factor_type='float_scale',
        bias_bit_width=32,
        weight_bit_width=BIT_WIDTH,
        weight_narrow_range=False,
        weight_param_method='stats',
        weight_quant_granularity='per_tensor',
        weight_quant_type='sym',
        layerwise_first_last_bit_width=BIT_WIDTH,
        act_bit_width=BIT_WIDTH,
        act_param_method='stats',
        act_quant_percentile=99.99,
        act_quant_type='sym',
        quant_format='int',
        layerwise_first_last_mantissa_bit_width=4,
        layerwise_first_last_exponent_bit_width=3,
        weight_mantissa_bit_width=4,
        weight_exponent_bit_width=3,
        act_mantissa_bit_width=4,
        act_exponent_bit_width=3).to(device) 

cpu


In [19]:
# from brevitas.graph.gpfq import gpfq_mode
# with gpfq_mode(quant_model,
#                p=1.0,
#                use_quant_activations=True,
#                act_order=False
#               ) as gpfq:
#             gpfq_model = gpfq.model  # Get the GPFQ-modified model

In [20]:
checkpoint_path = f'./DiscQuantResults/ModelParameter_Disc_{BIT_WIDTH}bit.pth'
checkpoint = torch.load(checkpoint_path)
quant_model.load_state_dict(checkpoint, strict=False)
device = torch.device("cuda")
quant_model=quant_model.to(device)

In [21]:
test_loss, test_acc = evaluate(quant_model, test_iterator, criterion)
print(f'\t Test. Loss: {test_loss:.3f} |  Test. Acc: {test_acc*100:.2f}%')

	 Test. Loss: 1.115 |  Test. Acc: 83.53%
